# 自定义函数（ [RunnableConfig](https://python.langchain.com/docs/expression_language/primitives/functions/)）

您可以在管道中使用任意函数。请注意，这些函数的所有输入都需要是单个参数。如果您有一个接受多个参数的函数，您应该编写一个包装器来接受单个输入并将其解包为多个参数。


In [7]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from langchain.globals import set_debug
set_debug(False) 

 `RunnableLambda`允许你将任意函数包装为一个`Runnable`，可以参与`pipeline`运算。传入`RunnableLambda`的函数必须只接受一个参数，如果本身是多参数的，要写一个`wrapper`，接受一个参数后解包传递。

  下面我们义`length_function`函数计算一个字符串的长度，`multiple_length_function`函数计算两个字符串长度的乘积。


In [8]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

chain.invoke({"foo": "bar", "bar": "gah"})


AIMessage(content='3 + 9 equals 12.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-43035dab-bdda-4760-85a2-4fa54736b1b6-0')

上述代码中，`chain`的第一步是一个字典，通过两层运算，将键a和b的值填充到prompt模板。比如对于键a：

- 使用`itemgetter`提取出需要计算的字符串"bar"
- 用`RunnableLambda`包装的函数计算字符串长度为3。
- 结果填充到`prompt`中，即a=3

  所以这段代码展示了如何在参数填充阶段，利用自定义函数进行复杂的数据预处理和计算，最终 Feed 到 prompting 的过程。整个链的关键就是插入了 `RunnableLambda enables` 我们插入自定义运算逻辑。


## Accepting a Runnable Config
`Runnable lambdas` 可以选择性地接受`RunnableConfig`，它们可以使用`RunnableConfig`将回调、标记和其他配置信息传递给嵌套运行。

In [9]:
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig, RunnableLambda

def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

In [10]:
from langchain_community.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

{'foo': 'bar'}
Tokens Used: 65
	Prompt Tokens: 56
	Completion Tokens: 9
Successful Requests: 1
Total Cost (USD): $0.00010200000000000001


## 流式自定义生成器函数
流式自定义生成器函数允许我们在`LCEL pipeline`中使用`yield`生成器函数作为自定义的输出解析器或中间处理步骤，同时保持流计算的特性。生成器函数的签名应该是`Iterator[Input] -> Iterator[Output]`，异步生成器应该是`AsyncIterator[Input] -> AsyncIterator[Output]`。

  流式自定义生成器函数主要有两个应用：

- 自定义输出解析器
- 不打破流计算的前提下处理流中的数据。
 
下面举一个示例，定义一个自定义的输出解析器`split_into_list`，来把`ChatGPT`的标记流解析为字符串列表。|


In [11]:
from typing import Iterator, List

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "Write a comma-separated list of 5 animals similar to: {animal}"
)
model = ChatOpenAI(temperature=0.0)
str_chain = prompt | model | StrOutputParser()

for chunk in str_chain.stream({"animal": "bear"}):
    print(chunk, end="", flush=True)  					# 输出：lion, tiger, wolf, gorilla, panda


1. Wolf
2. Tiger
3. Lion
4. Gorilla
5. Panda

或者是使用invoke方法：

In [ ]:
str_chain.invoke({"animal": "bear"})					# 输出：lion, tiger, wolf, gorilla, panda


下面定义一个自定义的解析器,将llm标记流分割成以逗号分隔的字符串列表

In [13]:
# 将输入的迭代器拆分成以逗号分隔的字符串列表
def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:    
    buffer = ""										# 保存部分输入直到遇到逗号
    
    for chunk in input:            					# 遍历输入的标记流迭代器input,每次将当前chunk添加到buffer
        buffer += chunk								             
        while "," in buffer:      					# 如果缓冲区中有逗号，获取逗号的索引              
            comma_index = buffer.index(",")    		                    
            yield [buffer[:comma_index].strip()]	# 生成逗号前的所有内容            
            buffer = buffer[comma_index + 1 :]		# 将逗号后面的内容保存给下一次迭代    
    yield [buffer.strip()]	


主要逻辑：

- 定义一个`buffer`字符串用于保存每次迭代读取的`chunk`
- 遍历输入的标记流迭代器`input`，每次将`chunk`添加到`buffer`
- 如果`buffer`中包含逗号，则
- 获取逗号的索引
- 将逗号前的内容取出，去空格后放进列表`yield`
- 将逗号后的内容留在`buffer`，等待下次迭代
- 最后一个`chunk`也做同样的解析`yield`出去


In [14]:
list_chain = str_chain | split_into_list
for chunk in list_chain.stream({"animal": "bear"}):
    print(chunk, flush=True)


['1. Wolf\n2. Lion\n3. Tiger\n4. Gorilla\n5. Panda']
